### Задача 1

In [122]:
!pip install yfinance
!pip install tslearn
!pip install minisom
!pip install tsfel
!pip install catboost
!pip install ta
!pip install catboost
!pip install feature_engine

In [123]:
import yfinance as yf
from sklearn.cluster import KMeans, DBSCAN
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset
from minisom import MiniSom
from ta import momentum, trend, volume
import pandas as pd
import tsfel

import csv

data = yf.download('AAPL', period='6mo', interval='1h')
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-03-14 09:30:00-04:00,172.320007,173.824997,172.050003,173.664993,173.664993,14937836
2024-03-14 10:30:00-04:00,173.669998,174.307800,173.300003,173.339996,173.339996,8727021
2024-03-14 11:30:00-04:00,173.345001,173.490005,172.399994,172.669998,172.669998,6752491
2024-03-14 12:30:00-04:00,172.679993,173.369202,172.240005,172.962997,172.962997,9628933
2024-03-14 13:30:00-04:00,172.968903,174.000000,172.899994,173.824997,173.824997,7128040
...,...,...,...,...,...,...
2024-09-13 11:30:00-04:00,223.544998,223.990005,223.054993,223.310104,223.310104,2825345
2024-09-13 12:30:00-04:00,223.320007,224.029999,223.220001,223.583801,223.583801,2326834
2024-09-13 13:30:00-04:00,223.580002,224.020004,222.850006,223.000000,223.000000,2945593


In [124]:
arr = []
with open('SP500.csv', 'r', newline='') as csvfile:                     #SP500.csv - название файла с тикерами акций
  spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
  for row in spamreader:
      arr.append(row[0].split(',')[0])
data = dict()
for stock_name in arr:
  try:
    elem = yf.download(stock_name, period='6mo', interval='1h')
  except:
    pass
  if len(elem) >= 876:
    data[len(data)] = elem
  if len(data) > 150:
    break


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['SYMBOL']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ATVI']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")')
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***

In [125]:
X, y = [], []
y1 = []
#В X считываются окна по 150 отметок (в данных есть 7 часовых отметок за рабочие сутки биржи, что в среднем соответствует 150 отметкам за 30 дней)
#В y считывается класс соответствующего окна - 1, если цена за следующие сутки выросла более чем на 2%, -1 если упала более чем на 2%, иначе 0
#В y1 считывается процентное изменение цены открытия за следующие сутки
for cnt in range(len(data)):
  d = data[cnt]
  for i in range(876-160):
    X.append([d['Open'].tolist()[i:i+150], d['High'].tolist()[i:i+150], d['Low'].tolist()[i:i+150], d['Close'].tolist()[i:i+150], d['Adj Close'].tolist()[i:i+150], d['Volume'].tolist()[i:i+150]])
    y1.append((d['Open'].tolist()[i+158] - d['Open'].tolist()[i+149]) / d['Open'].tolist()[i+149])
    if d['Open'].tolist()[i+158] > 1.02 * d['Open'].tolist()[i+149]:
      y.append(1)
    else:
      if d['Open'].tolist()[i+158] < 0.98 * d['Open'].tolist()[i+149]:
        y.append(-1)
      else:
        y.append(0)
#X = to_time_series_dataset(X)

Попробуем KMeans с евклидовым расстоянием и с метрикой DTW:

In [126]:
#X - трёхмерный тензор, А - его плоская версия для более удобной дальнейшей работы
X = np.array(X)
A = X.swapaxes(2, 1).reshape(X.shape[0], -1)

In [129]:
km_dba = TimeSeriesKMeans(n_clusters=4, metric="dtw", max_iter=10, n_jobs=3).fit(X)

In [130]:
cntt1, cntt2, cntt3, cntt4 = 0, 0, 0, 0
for i in range(len(A) // 626):
  cnt1, cnt2, cnt3, cnt4 = 0, 0, 0, 0
  for j in range(i * 626, i * 626 + 626):
    if km_dba.labels_[j] == 0:
      cnt1 += 1
      cntt1 += 1
    if km_dba.labels_[j] == 1:
      cnt2 += 1
      cntt2 += 1
    if km_dba.labels_[j] == 2:
      cnt3 += 1
      cntt3 += 1
    if km_dba.labels_[j] == 3:
      cnt4 += 1
      cntt4 += 1
  print(cnt1, cnt2, cnt3, cnt4)
print(cntt1, cntt2, cntt3, cntt4)

0 626 0 0
0 626 0 0
0 626 0 0
1 625 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 94 0 532
230 212 26 158
21 605 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
309 102 215 0
514 0 112 0
626 0 0 0
242 130 254 0
0 164 399 63
0 552 74 0
0 0 613 13
0 610 16 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
102 524 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 24 1 601
0 512 60 54
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 564 62 0
0 0 626 0
0 598 28 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 478 148 0
0 58 559 9
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
132 494 0 0
60 566 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
320 306 0 0
396 230 0 0
186 353 87 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
0 400 226 0
5 136 485 0
0 626 0 0
0 626 0 0
0 626 0 0
0 626 0 0
160 466 0 0
0 626 0 0
227 399 0 0
0 626 0 0
0 626 0 0

In [131]:
km_dba1 = TimeSeriesKMeans(n_clusters=4, metric="euclidean", max_iter=10, verbose=100, n_jobs=3).fit(X)
cntt1, cntt2, cntt3, cntt4 = 0, 0, 0, 0
for i in range(len(A) // 626):
  cnt1, cnt2, cnt3, cnt4 = 0, 0, 0, 0
  for j in range(i * 626, i * 626 + 626):
    if km_dba1.labels_[j] == 0:
      cnt1 += 1
      cntt1 += 1
    if km_dba1.labels_[j] == 1:
      cnt2 += 1
      cntt2 += 1
    if km_dba1.labels_[j] == 2:
      cnt3 += 1
      cntt3 += 1
    if km_dba1.labels_[j] == 3:
      cnt4 += 1
      cntt4 += 1
  print(cnt1, cnt2, cnt3, cnt4)
print(cntt1, cntt2, cntt3, cntt4)

212153242845223.188 --> 142540195227865.750 --> 141848541296828.188 --> 141622495454989.469 --> 141461350844226.219 --> 141372789784095.156 --> 141317168525666.781 --> 141282511688927.000 --> 141259198233012.281 --> 141249439842832.594 --> 
626 0 0 0
626 0 0 0
626 0 0 0
597 0 0 29
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
94 532 0 0
194 159 25 248
586 0 0 40
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
102 0 219 305
0 0 115 511
0 0 0 626
117 0 254 255
164 65 397 0
552 0 74 0
0 19 607 0
610 0 16 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
520 0 0 106
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
24 602 0 0
512 66 48 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
564 0 62 0
0 0 626 0
598 0 28 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
478 0 148 0
58 18 550 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
626 0 0 0
491 0 0 135
565 0 0 61
626 0 0 0
6

Теперь построим модель кластеризации, основанную на Self-Organizing Map:

In [132]:
som = MiniSom(4, 1, 900, sigma=0.1, learning_rate = 0.1, random_seed=0)

som.random_weights_init(A)
som.train(A, 500000)



In [133]:
cntt1, cntt2, cntt3, cntt4 = 0, 0, 0, 0
for i in range(len(A) // 626):
  cnt1, cnt2, cnt3, cnt4 = 0, 0, 0, 0
  for j in range(i * 626, i * 626 + 626):
    if som.winner(A[j])[0] == 0:
      cnt1 += 1
      cntt1 += 1
    if som.winner(A[j])[0] == 1:
      cnt2 += 1
      cntt2 += 1
    if som.winner(A[j])[0] == 2:
      cnt3 += 1
      cntt3 += 1
    if som.winner(A[j])[0] == 3:
      cnt4 += 1
      cntt4 += 1
  print(cnt1, cnt2, cnt3, cnt4, arr[i])
print(cntt1, cntt2, cntt3, cntt4)

0 626 0 0 Symbol
0 626 0 0 MMM
0 626 0 0 AOS
30 555 41 0 ABT
0 626 0 0 ABBV
0 626 0 0 ACN
0 626 0 0 ATVI
0 626 0 0 AYI
0 626 0 0 ADBE
0 94 0 532 AAP
143 122 177 184 AMD
41 498 87 0 AES
0 626 0 0 AET
0 626 0 0 AMG
0 626 0 0 AFL
0 626 0 0 A
0 626 0 0 APD
0 626 0 0 AKAM
0 626 0 0 ALK
0 626 0 0 ALB
0 626 0 0 ARE
0 626 0 0 ALXN
0 626 0 0 ALGN
0 626 0 0 ALLE
0 626 0 0 AGN
107 102 124 293 ADS
170 0 193 263 LNT
268 0 358 0 ALL
121 72 179 254 GOOGL
0 164 0 462 GOOG
0 552 0 74 MO
0 0 0 626 AMZN
0 610 0 16 AEE
0 626 0 0 AAL
0 626 0 0 AEP
0 626 0 0 AXP
0 626 0 0 AIG
0 626 0 0 AMT
0 626 0 0 AWK
0 626 0 0 AMP
0 626 0 0 ABC
0 624 2 0 AME
54 499 73 0 AMGN
0 626 0 0 APH
0 626 0 0 APC
26 564 36 0 ADI
0 626 0 0 ANDV
0 626 0 0 ANSS
0 24 0 602 ANTM
0 512 0 114 AON
7 608 11 0 APA
0 626 0 0 AIV
0 626 0 0 AAPL
0 626 0 0 AMAT
0 564 0 62 APTV
0 0 0 626 ADM
0 598 0 28 ARNC
0 626 0 0 AJG
0 626 0 0 AIZ
0 626 0 0 T
0 626 0 0 ADSK
0 478 0 148 ADP
0 58 0 568 AZO
0 626 0 0 AVB
0 626 0 0 AVY
0 626 0 0 BHGE
0 626 0 0 BL

В дальнейшем будем использовать KMeans с метрикой DTW, так как с кластерными метками от этого варианта кластеризации получаются наибольшие метрики качества.

In [134]:
#формирование основной части массива признаков XX

cfg_file1 = tsfel.get_features_by_domain("statistical")            
cfg_file2 = tsfel.get_features_by_domain("temporal")
XX = []
arr = []
#формирование массива дневных цен открытия, закрытия и т.д. для последующего расчёта технических индикаторов
X_days = []
for elem in X:
    new_elem1, new_elem2, new_elem3, new_elem4, new_elem5 = [], [], [], [], []
    for i in range(6):
        for j in range(len(elem[i])-7):
            if j % 7 == 0:
                new_elem1.append(elem[0][j])
                new_elem2.append(max(elem[1][j:j+7]))
                new_elem3.append(min(elem[2][j:j+7]))
                new_elem4.append(elem[3][j+7])
                new_elem5.append(sum(elem[5][j:j+7]))
    X_days.append([new_elem1, new_elem2, new_elem3, new_elem4, new_elem5])
for i in range(len(X)):
    new_elem = []
    # добавление технических индикаторов из библиотеки ta в массив признаков
    ind1 = momentum.AwesomeOscillatorIndicator(high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2])).awesome_oscillator()
    ind2 = momentum.KAMAIndicator(close=pd.Series(X_days[i][3])).kama()
    ind3 = momentum.RSIIndicator(close=pd.Series(X_days[i][3])).rsi()
    ind4 = momentum.StochRSIIndicator(close=pd.Series(X_days[i][3])).stochrsi()
    ind5 = trend.AroonIndicator(high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2])).aroon_indicator()
    ind6 = trend.MACD(close=pd.Series(X_days[i][3])).macd()
    ind7 = trend.MassIndex(high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2])).mass_index()
    ind8 = trend.aroon_up(high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2]))
    ind9 = trend.aroon_down(high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2]))
    ind10 = volume.ChaikinMoneyFlowIndicator(close=pd.Series(X_days[i][3]), high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2]), volume=pd.Series(X_days[i][4])).chaikin_money_flow()
    ind11 = volume.ease_of_movement(high=pd.Series(X_days[i][1]), low=pd.Series(X_days[i][2]), volume=pd.Series(X_days[i][4]))
    ind12 = km_dba.labels_[i]
    #ind12 = som.winner(A[i])[0]
    new_elem.append(ind1[len(ind1) - 1])
    new_elem.append(ind2[len(ind2) - 1])
    new_elem.append(ind3[len(ind3) - 1])
    new_elem.append(ind4[len(ind4) - 1])
    new_elem.append(ind5[len(ind5) - 1])
    new_elem.append(ind6[len(ind6) - 1])
    new_elem.append(ind7[len(ind7) - 1])
    new_elem.append(ind8[len(ind8) - 1])
    new_elem.append(ind9[len(ind9) - 1])
    new_elem.append(ind10[len(ind10) - 1])
    new_elem.append(ind11[len(ind11) - 1])
    new_elem.append(ind12)
    #добавление признаков временных рядов из библиотеки tsfel, спектральные признаки, вейвлеты и т.д. не используются 
    #в силу отсутствия выраженной сезонности колебаний в данных временных рядах
    X_ = tsfel.time_series_features_extractor(cfg_file1, X[i][0], fs=10, verbose=0)
    X2_ = tsfel.time_series_features_extractor(cfg_file2, X[i][0], fs=10, verbose=0)
    a = X_.iloc[0].tolist()
    for elem in X2_.iloc[0].tolist():
        a.append(elem)
    for elem in new_elem:
        a.append(elem)
    XX.append(a)
    #arr.append(ind)
    if i % 1000 == 0:
        print(i, len(X))                 #показывает прогресс формирования массива признаков
    

C:\Users\mixno\AppData\Local\Programs\Python\Python39\lib\site-packages\tsfel\feature_extraction\features.py:498: FutureWarning: The histogram feature was deprecated in version 0.1.8 and will be replaced by the mode of histogram in 0.1.9. From then on, only a single feature value will be returned.
  warnings.warn(


0 108116
1000 108116
2000 108116
3000 108116
4000 108116
5000 108116
6000 108116
7000 108116
8000 108116
9000 108116
10000 108116
11000 108116
12000 108116
13000 108116
14000 108116
15000 108116
16000 108116
17000 108116
18000 108116
19000 108116
20000 108116
21000 108116
22000 108116
23000 108116
24000 108116
25000 108116
26000 108116
27000 108116
28000 108116
29000 108116
30000 108116
31000 108116
32000 108116
33000 108116
34000 108116
35000 108116
36000 108116
37000 108116
38000 108116
39000 108116
40000 108116
41000 108116
42000 108116
43000 108116
44000 108116
45000 108116
46000 108116
47000 108116
48000 108116
49000 108116
50000 108116
51000 108116
52000 108116
53000 108116
54000 108116
55000 108116
56000 108116
57000 108116
58000 108116
59000 108116
60000 108116
61000 108116
62000 108116
63000 108116
64000 108116
65000 108116
66000 108116
67000 108116
68000 108116
69000 108116
70000 108116
71000 108116
72000 108116
73000 108116
74000 108116
75000 108116
76000 108116
77000 108116

In [135]:
# добавление технических индикаторов к массиву признаков
for i in range(len(XX)):
    ma7 = pd.Series(X_days[i][1]).rolling(window=7).mean()
    ma21 = pd.Series(X_days[i][1]).rolling(window=21).mean()

    # Create MACD
    ema26 = pd.Series(X_days[i][1]).ewm(span=26).mean()
    ema12 = pd.Series(X_days[i][1]).ewm(span=12).mean()
    MACD = ema12 - ema26

    # Create Bollinger Bands
    sd20 = pd.Series(X_days[i][1]).rolling(20).std()
    upper_band = ma21 + sd20*2
    lower_band = ma21 - sd20 * 2

    # Create Exponential moving average
    ema = pd.Series(X_days[i][1]).ewm(com=0.5).mean()

    # Create Momentum
    momentum = pd.Series(X_days[i][1])-1

    # Create log Momentum
    log_momentum = np.log(momentum)
    arr = [ma7, ma21, ema26, ema12, MACD, sd20, upper_band, lower_band, ema, momentum, log_momentum]
    for elem in arr:
        XX[i].append(elem[len(elem)-1])
    if i % 1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000


In [136]:
from sklearn.model_selection import train_test_split, cross_val_score
from catboost import CatBoostClassifier, cv, Pool
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, chi2, SelectPercentile
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from feature_engine.selection import DropCorrelatedFeatures


# Пробовал использовать два алгоритма для решения задачи классификации - случайный лес и CatBoost, RandomForestClassifier
# показывает метрики качества лучше
forest = RandomForestClassifier(n_estimators=70)                                           
forest.fit(XX, y)
selector = SelectFromModel(estimator=forest, threshold='1.2*median')              #отбор признаков по feature importance для случайного леса
XX_new = selector.transform(XX)
t = selector.get_support()
for i in range(len(t)):
    if t[i]:
        print(i, sep=' ')
print(len(XX_new[0]), len(XX_new[1]))
y = [(i, d) for i, d in enumerate(y)] 
XX_train, XX_test, y_train, y_test = train_test_split(
    XX_new, y, test_size=0.1, random_state=42, shuffle=True)
y1_ = [y1[y_test[i][0]] for i in range(len(y_test))]
y_train = [y_train[i][1] for i in range(len(y_train))]
y_test = [y_test[i][1] for i in range(len(y_test))]
y = [y[i][1] for i in range(len(y))]
arr1 = []
cnt1, cnt2, cnt3 = 0, 0, 0
XX_train2 = [elem for elem in XX_train]
y_train2 = [elem for elem in y_train]
for t in range(260000):                                                       #устранение классового дисбаланса путём дублирования окон, в которых
    var = np.random.randint(len(y_train2))                                    #происходит существенное изменение цены (>2%)
    if y_train2[var] != 0:
        XX_train2.append(XX_train2[var])
        y_train2.append(y_train2[var])
for elem in y_train2:
    if elem == -1:
        cnt3 += 1
    if elem == 0:
        cnt2 += 1
    if elem == 1:
        cnt1 += 1
print(cnt1, cnt2, cnt3)         #вывод на экран количества окон
model = RandomForestClassifier(n_estimators=150)
model.fit(XX_train2, y_train2)
#cross_val_score(model, X=XX, y=[y[elem] for elem in arr], cv=5, scoring='accuracy')
predict_test = model.predict(XX_test)
print(accuracy_score(predict_test, y_test))                                      #вывод на экран accuracy модели
total_income = 0
for i in range(len(predict_test)):
    total_income += predict_test[i] * y1_[i]
print(total_income / len(predict_test) * 100, "%")                       #"средняя доходность" в процентах от одного окна  

0
1
27
28
30
31
33
36
37
38
39
40
42
43
44
50
51
52
54
56
59
60
64
66
70
71
72
73
74
29 29
71820 76719 64139
0.8888272290048095
0.6221475492489776 %


In [138]:
from sklearn.metrics import precision_score, recall_score, fbeta_score
print(precision_score(model.predict(XX_test), y_test, average=None))                    #precision модели по классам 
print(recall_score(model.predict(XX_test), y_test, average=None))                       #recall модели по классам
print(fbeta_score(model.predict(XX_test), y_test, beta=1, average=None))                #f1 модели по классам

[0.73059361 0.93678295 0.68765957]
[0.75829384 0.92423192 0.73521383]
[0.74418605 0.93046512 0.71064204]


### Задача 2

In [141]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from catboost import CatBoostRegressor
from ta import momentum, trend, volume


#решается задача регрессии, были попробованы CatBoost, случайный лес и линейная регрессия - лучший результат показывает RandomForestRegressor

chunk_size = 350 # количество считываемых строк таблицы за одну итерацию цикла
time1 = 50 #количество таймстампов, которые учитываются при прогнозировании таргета 
time2 = 150 #количество таймстампов между последней строкой учитываемых данных и моментом, в который мы предсказываем таргет

#модель предсказывает таргет через 150 таймтстампов (в среднем 75 секунд) по 50 последним таймстампам (25 секунд)
chunks = pd.read_csv('MD/lob.csv', chunksize=chunk_size)        #в этой модели будем использовать только ордербук и фичи, сгенерированные из этих данных
cnt = 0
X, y = [], []
asks_prices_names, bids_prices_names = ['asks[' + str(i) + '].price' for i in range(25)], ['bids[' + str(i) + '].price' for i in range(25)]
asks_amounts_names, bids_amounts_names = ['asks[' + str(i) + '].amount' for i in range(25)], ['bids[' + str(i) + '].amount' for i in range(25)]
asks_prices, bids_prices, asks_amounts, bids_amounts = [], [], [], [] #списки с ценами на покупку/продажу 
n_active_columns = 2              #задаём число наилучших цен на покупку и продажу для одного таймстампа, которые будут использоваться в модели
for chunk in chunks:
    for i in range(25):
        asks_prices.append(chunk[asks_prices_names[i]].tolist())
        bids_prices.append(chunk[bids_prices_names[i]].tolist())
        asks_amounts.append(chunk[asks_amounts_names[i]].tolist())
        bids_amounts.append(chunk[bids_amounts_names[i]].tolist())
    for i in range(time1, chunk_size - time2):
        X.append([])
        for k in range(n_active_columns):
            for j in range(i - time1, i):
                X[-1].append(asks_prices[k][j])
            for j in range(i - time1, i):
                X[-1].append(bids_prices[k][j])
            for j in range(i - time1, i):
                X[-1].append(asks_amounts[k][j])
            for j in range(i - time1, i):
                X[-1].append(bids_amounts[k][j])
        a = asks_amounts[0][i + time2] * asks_prices[0][i + time2] + bids_amounts[0][i + time2] * bids_prices[0][i + time2]
        b = asks_amounts[0][i + time2] + bids_amounts[0][i + time2]
        target = a / b
        y.append(target)               # используем взвешенное среднее лучшей цены на покупку и лучшей цены на продажу как таргет
    if cnt < 100:
        pass
    else:
        break
    cnt += 1
XX = [[] for _ in range(len(X))]
#теперь перейдём к feature extraction
for i in range(len(X) - 1):
    elem = X[i]
    for j in range(time1):
        spread = elem[j] - elem[time1 + j]
        mid = (elem[j] + elem[time1 + j]) / 2
        price_difference_ask = elem[time1*4 + j] - elem[j]
        price_difference_bid = elem[time1*5 + j] - elem[time1 + j]
        price_mean = np.mean([elem[j + k * 4 * time1] for k in range(n_active_columns)])
        volume_mean = np.mean([elem[j + 2 * time1 + k * 4 * time1] for k in range(n_active_columns)])
        price_deviation = (X[i + 1][j] - X[i - 1][j]) #считаем разницу между соседними таймстампами слабо различающейся
        volume_deviation = (X[i + 1][j + 2*time1] - X[i - 1][j + 2*time1])
        #далее добавим фичи из технического анализа, воспользуемся библиотекой ta, как и в прошлом задании
        ind1 = momentum.AwesomeOscillatorIndicator(high=pd.Series(elem[:time1]), low=pd.Series(elem[time1:2*time1])).awesome_oscillator()
        ind2 = momentum.KAMAIndicator(close=pd.Series(elem[:time1])).kama()
        ind3 = momentum.RSIIndicator(close=pd.Series(elem[:time1])).rsi()
        ind4 = momentum.StochRSIIndicator(close=pd.Series(elem[:time1])).stochrsi()
        ind5 = trend.AroonIndicator(high=pd.Series(elem[:time1]), low=pd.Series(elem[time1:2*time1])).aroon_indicator()
        ind6 = trend.MACD(close=pd.Series(elem[:time1])).macd()
        ind7 = trend.MassIndex(high=pd.Series(elem[:time1]), low=pd.Series(elem[time1:2*time1])).mass_index()
        ind8 = trend.aroon_up(high=pd.Series(elem[:time1]), low=pd.Series(elem[time1:2*time1]))
        ind9 = trend.aroon_down(high=pd.Series(elem[:time1]), low=pd.Series(elem[time1:2*time1]))
        ind11 = volume.ease_of_movement(high=pd.Series(elem[:time1]), low=pd.Series(elem[time1:2*time1]), volume=pd.Series(elem[2*time1:3*time1]))
        XX[i].append(ind1[len(ind1) - 1])
        XX[i].append(ind2[len(ind2) - 1])
        XX[i].append(ind3[len(ind3) - 1])
        XX[i].append(ind4[len(ind4) - 1])
        XX[i].append(ind5[len(ind5) - 1])
        XX[i].append(ind6[len(ind6) - 1])
        XX[i].append(ind7[len(ind7) - 1])
        XX[i].append(ind8[len(ind8) - 1])
        XX[i].append(ind9[len(ind9) - 1])
        #XX[i].append(ind10[len(ind10) - 1])
        XX[i].append(ind11[len(ind11) - 1])
        #добавим также часть самостоятельно реализованных признаков, как и в предыдущей задаче:
        ma7 = pd.Series(elem[:time1]).rolling(window=7).mean()
        ma21 = pd.Series(elem[:time1]).rolling(window=21).mean()
    
        # Create MACD
        ema26 = pd.Series(elem[:time1]).ewm(span=26).mean()
        ema12 = pd.Series(elem[:time1]).ewm(span=12).mean()
        MACD = ema12 - ema26
    
        # Create Bollinger Bands
        sd20 = pd.Series(elem[:time1]).rolling(20).std()
        upper_band = ma21 + sd20*2
        lower_band = ma21 - sd20 * 2
    
        # Create Exponential moving average
        ema = pd.Series(elem[:time1]).ewm(com=0.5).mean()
    
        arr = [ma7, ma21, ema26, ema12, MACD, sd20, upper_band, lower_band, ema, spread, mid, price_difference_ask, price_difference_bid, price_mean, volume_mean, price_deviation, volume_deviation]
        for el in arr:
            try:
                el[0]
                XX[i].append(el[len(el)-1])
            except:
                XX[i].append(el)
forest = RandomForestRegressor(max_depth=10, n_estimators=90)
XX_train, XX_test, y_train, y_test = train_test_split(
    XX[:-1], y[:-1], test_size=0.2, random_state=42, shuffle=False)
forest.fit(XX_train, y_train)
mean_absolute_error(forest.predict(XX_test), y_test) / np.std(y_test)             #используем отношение MAE к стандартному отклонению таргета
                                                                                  #как метрику качества


0.0017650479458726828